In [1]:
# 导入必要的库
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import torch
import numpy as np
import random
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
import torch.nn.functional as F

In [2]:
seed=42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
num_class=2
adj_parameter = 10
dim_he_list = [200,200,100]
gcn_dopout=0.5
classifier_out_dim=2
num_epoch=500
test_inverval=20

In [4]:
# 读取制表符分隔的CSV文件
Methylation_train = pd.read_csv('../../DATA/BRCA/Methylation_train.csv')
# 显示 DataFrame
Methylation_train

,PC1_row_0,PC2_row_0,PC3_row_0,PC4_row_0,PC5_row_0,PC6_row_0,PC7_row_0,PC8_row_0,PC9_row_0,PC10_row_0,...,PC16_row_2760,PC17_row_2760,PC18_row_2760,PC19_row_2760,PC20_row_2760,PC21_row_2760,PC22_row_2760,PC23_row_2760,PC24_row_2760,PC25_row_2760
0,-1.133112,0.432254,-0.752333,0.021332,-0.076750,0.136673,-0.056860,-0.129899,0.022071,0.197129,...,-0.129697,0.110355,0.092455,-0.004343,0.007438,0.061179,-0.019424,-0.035152,-0.016164,0.015685
1,-0.328272,-0.327782,-0.260210,-0.138131,-0.504069,-0.209471,-0.551203,0.005745,-0.085503,-0.289122,...,-0.047125,0.029898,0.085592,0.028998,-0.091725,0.052117,0.011698,0.030157,0.034254,0.046613
2,0.972598,-0.765660,-0.629249,1.137107,0.869088,-0.106976,-0.994815,0.354060,0.429835,-0.367123,...,0.158909,0.164448,-0.115960,-0.182981,0.067097,-0.001231,-0.029327,-0.065763,-0.052208,-0.000725
3,-0.563105,-0.275874,-0.443432,-0.585330,-0.308654,0.083149,-0.030949,0.108816,0.274321,-0.039437,...,0.048716,-0.104846,-0.041441,0.072681,-0.077928,-0.049505,-0.201503,0.002145,-0.002652,0.013819
4,-1.273712,-0.137526,-1.126991,0.467694,-0.006499,-0.272419,-0.420170,0.045502,-0.009221,0.110928,...,-0.084024,0.090336,0.088669,-0.070569,-0.084756,-0.074723,0.044426,-0.051913,0.138375,-0.039888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,-0.510975,0.562041,-0.706093,-0.197077,0.315839,0.234255,0.302738,-0.090402,0.023627,0.164116,...,-0.022434,0.018172,0.010903,-0.017678,-0.013426,0.029170,-0.037155,0.009093,-0.040714,0.069887
479,0.619017,-1.298486,-0.178894,0.279145,-0.032568,1.127893,0.292049,0.194871,-0.488433,-0.183756,...,0.220085,0.096371,0.305330,0.006572,0.333682,0.146095,-0.152645,-0.036693,-0.036303,-0.027846
480,0.177790,0.069162,-0.385319,-0.327666,0.429933,0.733198,-0.059441,0.261967,0.037745,-0.215350,...,-0.004345,-0.044623,0.005419,-0.058057,-0.038374,0.003122,-0.041482,0.000553,0.094309,0.026549
481,-2.680920,-1.107335,0.396566,0.602481,-0.296157,0.221798,0.324225,0.021138,0.059036,-0.041427,...,-0.163104,-0.051909,0.122783,-0.113140,-0.122921,-0.067630,0.021647,-0.004833,-0.057791,-0.033498


In [5]:
# 读取制表符分隔的CSV文件
Methylation_test = pd.read_csv('../../DATA/BRCA/Methylation_test.csv')
# 显示 DataFrame
Methylation_test

,PC1_row_0,PC2_row_0,PC3_row_0,PC4_row_0,PC5_row_0,PC6_row_0,PC7_row_0,PC8_row_0,PC9_row_0,PC10_row_0,...,PC16_row_2760,PC17_row_2760,PC18_row_2760,PC19_row_2760,PC20_row_2760,PC21_row_2760,PC22_row_2760,PC23_row_2760,PC24_row_2760,PC25_row_2760
0,-0.083057,-0.453855,-0.355854,-0.597989,-0.227851,-0.255867,-0.067971,-0.108719,0.012229,-0.002228,...,-0.083241,0.000866,-0.012588,-0.059448,0.005727,0.060929,0.022743,0.065583,0.032022,0.037633
1,-0.116828,0.208500,-0.281834,0.078507,-0.326000,-0.129113,0.389942,0.284545,-0.549864,0.065550,...,0.138913,-0.060757,-0.062266,-0.011352,-0.136438,0.219568,0.151907,0.008193,-0.044848,-0.008064
2,3.026874,1.360013,1.604133,0.301679,-0.184280,1.031975,-0.347907,-0.376378,0.079896,0.414958,...,0.113207,0.040958,-0.171555,0.104158,-0.055427,-0.158649,-0.108100,-0.233674,0.187397,0.394893
3,0.557285,-0.433981,-0.174698,-0.744424,0.247322,-0.469112,-0.101189,-0.512527,-0.005555,0.474726,...,-0.112917,-0.081183,-0.019158,-0.068685,-0.018085,0.158199,0.051271,0.041618,-0.048401,0.063796
4,-1.595965,0.093374,0.977229,0.677707,-0.752134,-0.209670,0.143848,-0.078581,-0.063570,0.178098,...,0.148706,-0.008215,-0.088051,-0.086524,-0.188028,0.059614,0.126860,0.108724,-0.004557,-0.007943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,-2.493303,-0.622984,1.128149,-0.021780,0.460668,0.250298,-0.611954,-0.547334,0.061195,0.071800,...,0.048022,-0.186728,-0.055183,0.001080,-0.121086,-0.045344,0.079301,-0.039372,0.092575,-0.049881
147,-0.432135,1.021195,-0.568258,-0.161315,0.001845,0.162578,0.032326,-0.157578,-0.211705,-0.112964,...,-0.060662,0.046584,0.028403,0.034945,0.097208,0.073732,-0.051729,-0.012437,0.002975,0.028298
148,-0.829848,-0.351822,-0.885172,0.598137,0.866581,-0.351287,0.305799,0.069113,0.404125,-0.063066,...,-0.208781,-0.006602,-0.062632,-0.033484,-0.089766,0.067176,0.099999,0.007793,0.013101,0.072729
149,1.817912,0.394121,0.381914,0.051661,0.390210,-0.506131,-0.036802,-0.072685,0.157694,-0.040606,...,0.122984,-0.038678,0.026806,0.079602,-0.018049,-0.039122,0.038672,0.014961,-0.054452,0.030638


In [6]:
# 读取制表符分隔的CSV文件
Methylation_val = pd.read_csv('../../DATA/BRCA/Methylation_val.csv')
# 显示 DataFrame
Methylation_val

,PC1_row_0,PC2_row_0,PC3_row_0,PC4_row_0,PC5_row_0,PC6_row_0,PC7_row_0,PC8_row_0,PC9_row_0,PC10_row_0,...,PC16_row_2760,PC17_row_2760,PC18_row_2760,PC19_row_2760,PC20_row_2760,PC21_row_2760,PC22_row_2760,PC23_row_2760,PC24_row_2760,PC25_row_2760
0,2.052949,-0.513379,0.189703,-0.864970,0.139891,0.110523,0.656754,0.238135,-0.057738,0.020201,...,0.084104,0.091539,-0.077762,-0.077746,0.059516,-0.085020,-0.050145,0.060589,-0.052135,-0.032661
1,-0.099362,-0.883941,-0.395110,-0.868775,-0.450819,0.272821,-0.286715,0.672576,0.007189,-0.173691,...,0.083478,0.025237,-0.208883,0.070958,0.007943,-0.021710,-0.124033,-0.001385,-0.046073,-0.073962
2,-0.795682,0.919179,-0.739628,-0.021590,0.059862,0.238586,-0.061212,-0.147995,-0.057729,-0.209544,...,-0.021236,0.063365,-0.007381,-0.018240,-0.078170,0.081476,-0.015207,-0.017921,0.055032,-0.027465
3,-1.591239,1.890452,-0.705911,-0.475315,0.208076,0.329394,0.542322,-0.011635,0.274357,-0.254059,...,-0.081009,-0.006619,-0.100247,0.016510,0.050821,-0.048949,-0.010425,-0.073211,0.055090,-0.059167
4,-1.027266,0.003539,-0.692147,-0.060090,-0.455979,-0.041210,-0.359007,-0.476660,-0.121044,0.289945,...,-0.013821,-0.033014,0.030023,-0.036485,-0.042313,0.049548,0.030879,0.003942,0.039844,0.004409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,1.429551,0.212816,0.257470,-0.182585,0.565436,0.592500,-0.272069,0.301166,0.417191,-0.507885,...,0.075888,-0.113991,-0.039005,0.154038,0.092205,0.031804,0.027915,-0.060485,0.037225,-0.087834
117,-0.135944,0.412549,-0.634014,0.020806,-0.323595,-0.477751,-0.183023,0.030702,0.137273,-0.102008,...,0.003500,-0.000416,0.199974,-0.042842,-0.097216,0.053882,0.040794,-0.008000,0.163776,0.024381
118,-0.251558,-0.437769,-0.441844,-0.720386,-0.251123,0.206201,-0.270221,-0.175612,-0.196000,0.520840,...,-0.019545,-0.052196,-0.011517,0.010289,-0.116807,0.105970,0.046690,0.016135,-0.039427,0.017190
119,-1.375624,1.640514,-0.683941,-0.146128,0.053231,0.277419,0.735907,0.001520,0.145800,-0.243926,...,-0.049064,-0.140096,-0.098356,-0.007793,-0.060562,-0.057282,0.057979,-0.002347,0.130465,0.015768


In [7]:
# 读取制表符分隔的CSV文件
SCNA_train = pd.read_csv('../../DATA/BRCA/SCNA_train.csv')
# 显示 DataFrame
SCNA_train

,PC1_row_0,PC2_row_0,PC3_row_0,PC4_row_0,PC5_row_0,PC6_row_0,PC7_row_0,PC8_row_0,PC9_row_0,PC10_row_0,...,PC46_row_2056,PC47_row_2056,PC48_row_2056,PC49_row_2056,PC50_row_2056,PC51_row_2056,PC52_row_2056,PC53_row_2056,PC54_row_2056,PC55_row_2056
0,-3.042972,-0.036396,0.856437,0.306609,-0.030241,0.249793,-0.868286,-0.229788,-0.651587,-0.191455,...,0.039366,-0.000533,-0.116374,-0.015636,-0.008717,-0.024181,-0.049676,-0.235084,0.071269,-0.164961
1,-1.308227,-0.530738,0.858029,-1.455780,-0.851286,-0.553634,-0.629438,-0.078062,-0.107059,0.294721,...,0.023492,-0.006950,0.015144,-0.036227,0.053869,-0.028833,0.012355,0.048827,-0.038583,-0.030644
2,1.903435,1.015962,-1.696413,-0.814995,0.487137,-1.848661,0.837131,-1.705378,0.152750,-0.965777,...,-0.373319,-0.183891,-0.214860,-0.102975,0.240462,0.008817,-0.056475,0.182479,-0.152387,-0.095265
3,0.477532,-2.600101,-0.139348,-0.316355,-0.689007,1.057880,-0.792376,0.202498,-1.542821,-0.060008,...,-0.013222,0.043793,0.123015,-0.007974,-0.137699,0.028201,-0.065576,0.017748,0.044716,-0.019196
4,-3.780153,-0.608201,0.307205,0.200422,-1.050966,-0.073396,0.216174,0.190776,-0.185202,0.264273,...,-0.007965,-0.043995,0.015668,0.072441,0.124781,0.034010,0.014295,-0.044076,-0.016793,-0.044668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,-2.761458,1.467455,0.622201,-0.412402,-2.218062,0.210539,-0.321246,0.763132,0.422586,0.175001,...,0.008656,-0.095337,0.075150,-0.024180,0.060402,0.011209,-0.031096,0.030246,0.034062,-0.048628
479,-4.168434,-0.588783,0.789643,0.782460,-0.864059,0.337763,-0.556786,-0.229154,-0.466455,-0.350507,...,0.057056,0.033941,-0.017594,-0.060292,0.046236,0.023075,-0.027446,-0.060373,-0.113437,-0.000233
480,-3.726883,-0.619672,0.963800,0.734945,-1.103897,1.249145,-0.549713,0.240739,-0.404901,0.689113,...,-0.088782,0.168266,-0.063875,0.050816,0.051492,0.107765,-0.035484,-0.100441,-0.007207,0.021043
481,4.702578,0.491372,6.241917,2.948750,-1.016198,2.250271,-2.364181,-0.492034,0.964039,-1.456188,...,0.007004,0.186498,0.146128,-0.431444,-0.425958,0.450790,0.360507,-0.637586,-0.079015,-0.273235


In [8]:
# 读取制表符分隔的CSV文件
SCNA_test = pd.read_csv('../../DATA/BRCA/SCNA_test.csv')
# 显示 DataFrame
SCNA_test

,PC1_row_0,PC2_row_0,PC3_row_0,PC4_row_0,PC5_row_0,PC6_row_0,PC7_row_0,PC8_row_0,PC9_row_0,PC10_row_0,...,PC46_row_2056,PC47_row_2056,PC48_row_2056,PC49_row_2056,PC50_row_2056,PC51_row_2056,PC52_row_2056,PC53_row_2056,PC54_row_2056,PC55_row_2056
0,1.164760,-3.057396,-0.793069,-0.347218,-0.428860,-0.660877,-1.356888,-0.726705,0.409955,-0.454943,...,-0.121960,-0.255794,0.080046,0.223597,-0.007488,0.000744,0.163740,0.237797,0.664433,-0.015983
1,-4.326218,-0.399924,-0.024923,-2.488456,1.033744,-1.125590,-0.822251,1.401116,0.304170,0.647103,...,-0.257182,0.164668,-0.063891,0.058931,-0.129091,-0.030706,0.041322,0.020549,0.094145,-0.034296
2,4.470705,4.129240,1.944123,0.794482,-0.229099,-0.276505,1.049322,-1.581868,-0.953439,0.908932,...,-0.050772,0.105961,0.126961,0.160245,0.157073,0.050528,0.189868,0.079189,-0.077877,0.129964
3,-0.279167,-0.030789,0.281411,-0.476994,-0.536856,-1.254177,-0.457786,-0.326185,-0.015293,-0.974720,...,0.124842,-0.061104,0.201472,-0.133711,-0.041391,-0.125697,0.102954,-0.171770,0.056361,-0.001137
4,-0.313617,-0.823879,1.454771,-2.009039,4.152889,-0.661828,-0.419727,0.404559,-0.187676,1.151941,...,-0.434818,0.056019,-0.312778,0.051735,-0.026207,-0.057249,-0.233163,-0.203178,-0.447222,-0.163997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,4.810888,1.698030,4.784220,1.585261,0.208319,4.923175,1.240104,-1.484503,-2.336960,-2.804314,...,0.614570,-0.094146,0.560739,-0.199795,0.438984,-0.276397,0.208881,-0.423141,-1.056117,-0.142896
147,-2.336340,0.355663,-1.686122,3.697391,0.057012,-0.390222,-0.494656,1.256590,-1.424609,-0.893492,...,0.010522,-0.103537,0.085481,0.043028,-0.087596,0.025023,0.180352,0.070812,-0.184872,-0.155515
148,2.500678,4.772001,2.019497,-1.510510,-2.445919,-1.766254,-0.077504,0.241256,0.298330,0.386723,...,-0.468374,0.134249,0.056758,0.423887,-0.036064,-0.104988,0.041207,0.011600,-0.145239,0.054745
149,2.722790,1.898408,-2.252526,-3.157469,2.140439,-1.757007,1.034696,0.938212,0.049647,-1.515088,...,-0.065673,0.072405,0.369211,-0.303678,0.165591,-0.230093,-0.099729,0.311280,-0.164900,0.106788


In [9]:
# 读取制表符分隔的CSV文件
SCNA_val = pd.read_csv('../../DATA/BRCA/SCNA_val.csv')
# 显示 DataFrame
SCNA_val

,PC1_row_0,PC2_row_0,PC3_row_0,PC4_row_0,PC5_row_0,PC6_row_0,PC7_row_0,PC8_row_0,PC9_row_0,PC10_row_0,...,PC46_row_2056,PC47_row_2056,PC48_row_2056,PC49_row_2056,PC50_row_2056,PC51_row_2056,PC52_row_2056,PC53_row_2056,PC54_row_2056,PC55_row_2056
0,4.607736,-2.007509,-0.283876,-1.369454,-0.484090,-0.945664,-1.139873,0.379082,0.726747,-1.515274,...,0.021082,0.017338,0.029177,0.023125,0.037338,-0.061847,0.017762,-0.015938,-0.038285,-0.048545
1,-0.751290,-2.013462,-0.116436,-0.366908,-0.411289,-0.799698,-1.036693,-0.614901,-1.156902,-0.430573,...,0.137563,-0.073045,-0.046625,-0.013768,0.055715,0.034005,0.008817,0.041883,-0.023613,0.001465
2,-3.119487,-0.076017,0.761189,0.399079,-0.456862,0.289822,-0.554403,-0.463034,-0.788896,-0.101898,...,0.003160,0.067372,-0.049747,-0.032481,-0.012953,0.033088,-0.050300,0.073043,-0.019793,-0.072790
3,-1.975989,-0.876953,1.424446,0.736183,-0.496554,0.377811,-0.899689,-0.342995,-0.916170,-0.075578,...,-0.127251,0.001376,0.018014,-0.055137,-0.086393,-0.025750,0.079170,-0.064593,0.030237,0.077522
4,-3.416277,-0.944033,0.860297,0.993687,-1.530462,0.099947,-0.238545,-0.534648,-0.108028,-0.314083,...,-0.025800,-0.028036,0.023286,0.043282,0.070756,-0.012361,0.015124,-0.013743,-0.007773,0.006621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,5.311199,0.126521,-0.041265,-2.463835,0.369680,-3.123892,-1.211740,-0.978819,-1.699411,1.109903,...,0.182672,-0.020747,-0.220164,0.198354,0.226449,-0.134484,-0.224305,-0.250644,0.115127,-0.272378
117,-0.923477,-1.377378,-0.643848,0.492647,-0.784617,0.543551,-0.504212,-0.070519,0.019597,0.210534,...,0.032871,0.005919,0.195066,0.024223,0.034545,-0.052188,0.126654,-0.237293,0.017103,0.085860
118,1.094540,-1.698069,-0.286114,-0.479389,-0.558388,-0.306297,-0.270099,-0.965184,-0.496581,-0.885717,...,-0.128941,-0.199854,-0.062309,-0.151940,-0.116174,-0.041813,-0.055270,0.097874,-0.115140,0.048766
119,-2.999129,-0.890075,1.480538,1.022761,-0.494227,0.914500,0.002563,-0.076068,-0.302347,-0.420255,...,-0.028551,-0.016752,-0.053808,0.043153,-0.025519,0.023438,0.014501,-0.060623,-0.079268,0.025349


In [10]:
# 读取制表符分隔的CSV文件
RNAseq_train = pd.read_csv('../../DATA/BRCA/RNAseq_train.csv')
# 显示 DataFrame
RNAseq_train

,PC1_row_0,PC2_row_0,PC3_row_0,PC4_row_0,PC5_row_0,PC6_row_0,PC7_row_0,PC8_row_0,PC9_row_0,PC10_row_0,...,PC15_row_2897,PC16_row_2897,PC17_row_2897,PC18_row_2897,PC19_row_2897,PC20_row_2897,PC21_row_2897,PC22_row_2897,PC23_row_2897,PC24_row_2897
0,5.722480,-9.931543,6.266378,-0.399020,-1.362642,-3.341852,2.983187,-1.945759,2.251672,3.242442,...,-0.331674,-0.160858,-0.413210,-0.295111,0.100677,-0.487805,0.134342,-0.185612,0.252541,0.186363
1,2.021931,-11.878873,11.471604,6.210857,-0.655655,-1.880423,0.406555,0.253020,-0.535813,-0.521891,...,-0.642033,-0.158217,-0.639091,0.253332,-0.379835,-0.071436,0.494031,0.245754,0.338458,0.281539
2,-7.696640,3.591070,0.507967,3.879902,-6.219841,-6.668944,-5.102602,-1.845159,4.793140,-3.129187,...,-2.039214,0.560806,0.178317,0.185900,-0.417199,0.866212,-0.820033,0.535724,0.451224,-0.303397
3,-1.964178,-7.729450,-3.272530,3.349683,4.181307,0.600221,-2.711520,-0.465703,-0.710711,1.926945,...,-0.277486,0.005084,-0.017699,-0.142254,-0.882170,-0.109919,0.195118,0.283755,-0.722930,-0.099915
4,-0.188483,-9.126740,-3.454491,-8.239586,6.025050,-0.290525,-5.047860,1.765638,3.018912,1.975052,...,-0.056473,0.110871,0.025619,-0.689282,0.284933,0.289510,-0.504005,0.435911,-0.102841,0.445917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,-1.788636,-9.416392,-0.215939,0.772592,-0.690191,-1.675779,0.553424,-2.871225,-1.040378,-0.377929,...,-0.233011,0.586583,-0.289079,0.064345,-0.399968,0.009789,-0.134570,-0.131345,0.168194,0.311981
479,-13.432338,13.385199,0.725380,5.384704,12.983777,-4.244443,8.802238,12.536842,6.394253,2.946900,...,0.876839,-0.775164,-0.170267,1.664434,0.181409,-0.088797,1.155538,0.135126,-0.875742,-0.066844
480,-6.263201,8.652268,1.565184,1.112054,5.633352,-0.450290,4.851715,-2.839895,7.464113,4.836615,...,0.768000,-0.069334,0.305978,0.956131,-0.272339,-0.010615,0.266817,-0.107206,-0.723125,-0.029091
481,17.312951,0.199890,-8.424050,7.083630,3.344161,4.932113,-1.154465,1.487478,0.928929,5.497423,...,-0.311394,-0.847188,0.120238,-1.036224,1.885884,0.161237,0.211313,0.071067,-1.127790,0.679642


In [11]:
# 读取制表符分隔的CSV文件
RNAseq_test = pd.read_csv('../../DATA/BRCA/RNAseq_test.csv')
# 显示 DataFrame
RNAseq_test

,PC1_row_0,PC2_row_0,PC3_row_0,PC4_row_0,PC5_row_0,PC6_row_0,PC7_row_0,PC8_row_0,PC9_row_0,PC10_row_0,...,PC15_row_2897,PC16_row_2897,PC17_row_2897,PC18_row_2897,PC19_row_2897,PC20_row_2897,PC21_row_2897,PC22_row_2897,PC23_row_2897,PC24_row_2897
0,-1.432554,-16.927542,-3.308566,2.326964,0.852780,-2.500608,-1.822288,1.609805,-1.390099,-0.459052,...,0.730766,0.010611,0.002241,-0.598237,0.131939,-0.342714,-0.563395,-0.410972,-1.036643,0.425115
1,-6.418740,-4.343279,-1.162348,-8.670454,-3.105773,1.221421,1.440550,-1.885588,-3.884798,1.806159,...,-0.164879,0.310869,-0.454520,0.290026,0.801253,0.581336,0.234538,0.377186,-0.707260,-0.021210
2,-9.370020,15.672699,-4.850974,6.809162,-9.433864,-8.451102,4.787379,2.450430,4.892452,-2.583510,...,-1.336881,0.369441,-1.762382,0.434057,-0.387933,-0.489589,-0.554258,0.350991,-0.274582,-0.472173
3,-4.696742,-7.281025,2.425139,2.094841,-1.341783,-0.841350,-1.695552,1.719991,-1.820749,6.078094,...,0.114167,-0.545820,0.251931,-0.208312,0.143704,0.635113,0.468139,-0.388129,0.007241,-0.558883
4,20.812070,4.635553,2.481615,-0.851986,-1.310346,-1.923644,-3.978962,-1.401079,2.538861,-3.229553,...,-0.306097,0.570986,-0.469502,-0.424874,-0.095480,0.408010,-0.464095,-0.047846,-0.541181,-0.069989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,23.049708,-5.946812,-3.062978,-0.205704,0.538790,-5.867978,2.358093,6.839584,-11.449366,3.084016,...,1.378005,0.793871,0.618350,0.095627,-0.007300,0.944003,-0.553948,-0.786351,0.162892,-0.043143
147,1.192625,-12.521053,-2.592417,3.452987,-3.953844,-0.119069,4.769488,-3.072535,2.612417,-2.390248,...,0.272874,0.611903,-0.226567,0.474251,0.022137,-0.286307,-0.081187,0.682460,0.290174,-0.116565
148,-6.317248,-0.260888,-0.705914,-4.032808,-1.416835,-2.406623,1.283297,-4.439875,-1.619425,-1.243601,...,-0.760535,-0.928787,0.615954,1.550821,0.757052,-0.112186,0.465988,-0.493423,-0.012688,0.735859
149,-9.782726,3.352288,-0.391029,-2.797746,-5.558714,-5.767338,-3.508455,-0.552711,-1.808577,4.036865,...,-0.928206,-0.373738,0.130774,-0.248497,-0.083983,0.577544,-0.215349,0.446654,-0.185512,0.710806


In [12]:
# 读取制表符分隔的CSV文件
RNAseq_val = pd.read_csv('../../DATA/BRCA/RNAseq_val.csv')
# 显示 DataFrame
RNAseq_val

,PC1_row_0,PC2_row_0,PC3_row_0,PC4_row_0,PC5_row_0,PC6_row_0,PC7_row_0,PC8_row_0,PC9_row_0,PC10_row_0,...,PC15_row_2897,PC16_row_2897,PC17_row_2897,PC18_row_2897,PC19_row_2897,PC20_row_2897,PC21_row_2897,PC22_row_2897,PC23_row_2897,PC24_row_2897
0,-13.908132,7.354229,-6.507595,1.237615,9.109518,1.300425,-7.264757,-4.403238,5.117670,1.006509,...,2.173017,0.716473,-0.572706,-0.444462,0.281522,-0.669105,0.145635,-0.368956,-0.016783,-0.681784
1,1.271496,-13.729955,6.174848,6.231470,0.457016,7.397776,-1.282181,4.474308,-1.891516,5.243326,...,1.218511,-0.418126,0.415327,0.186007,-0.662267,0.133288,0.238856,-0.191328,-0.514653,0.599419
2,1.994857,-8.142788,-3.818002,2.512522,-4.547672,-1.712753,4.344315,-6.302578,4.896869,0.582922,...,-0.337517,0.195081,0.603464,0.400138,0.119630,0.125041,-0.265997,-0.548410,-0.556736,0.035124
3,17.362630,-1.173860,-2.437287,-0.955201,2.881060,-1.033082,4.705226,-6.234226,1.516760,1.302675,...,-0.302409,0.789252,-0.686131,0.607113,0.209433,-0.211333,0.174363,-0.185250,0.024094,0.344903
4,5.486387,-17.809487,-4.173803,2.990161,0.467975,0.104486,-2.504349,3.178914,3.621787,-3.835961,...,0.807522,0.068498,-0.013774,0.152634,-0.509474,-0.261142,-0.255416,-0.473271,-0.029182,-0.472885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,-11.711927,15.477495,-3.932902,5.789284,0.380440,6.931645,3.883790,-2.059833,4.175447,8.664329,...,-0.864989,-1.571656,-0.722532,0.910188,-0.445002,0.888404,0.464853,-0.021268,-0.268396,-0.147423
117,2.415974,-13.206339,2.936998,-2.490859,4.677418,1.568958,-3.999002,0.209672,-2.158525,-1.804573,...,-0.584233,-0.279002,-0.341447,-0.224328,0.304620,0.304317,-0.177682,0.493944,0.009001,-0.220227
118,-3.556367,-9.277939,-1.824367,6.648340,3.785980,-0.113046,-0.600704,-1.397897,-1.000215,-3.645757,...,0.419200,-0.561647,0.448392,0.575112,0.258757,-0.574999,-0.019422,-0.161496,-0.789062,0.208007
119,12.209748,-4.200269,-0.880719,1.524398,0.233670,2.917874,3.053611,-5.687430,0.249969,-3.531847,...,-0.453508,-0.011068,0.020481,-0.825179,0.330664,0.353666,-0.333428,-0.130037,-0.074286,0.593438


In [13]:
labels_train = pd.read_csv('../../DATA/BRCA/labels_train.csv')
labels_train

,status
0,0.0
1,0.0
2,0.0
3,0.0
4,1.0
...,...
478,0.0
479,0.0
480,0.0
481,0.0


In [14]:
labels_test = pd.read_csv('../../DATA/BRCA/labels_test.csv')
labels_test

,status
0,0.0
1,1.0
2,0.0
3,1.0
4,0.0
...,...
146,0.0
147,0.0
148,0.0
149,1.0


In [15]:
# 读取制表符分隔的CSV文件
labels_val = pd.read_csv('../../DATA/BRCA/labels_val.csv')
# 显示 DataFrame
labels_val

,status
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
116,0.0
117,0.0
118,0.0
119,0.0


In [16]:
def cosine_distance_torch(x1, x2=None, eps=1e-8):
    x2 = x1 if x2 is None else x2
    w1 = x1.norm(p=2, dim=1, keepdim=True)
    w2 = w1 if x2 is x1 else x2.norm(p=2, dim=1, keepdim=True)
    return 1 - torch.mm(x1, x2.t()) / (w1 * w2.t()).clamp(min=eps)

In [17]:
def cal_adj_mat_parameter(edge_per_node, data, metric="cosine"):
    assert metric == "cosine", "Only cosine distance implemented"
    dist = cosine_distance_torch(data, data)
    parameter = torch.sort(dist.reshape(-1,)).values[edge_per_node*data.shape[0]]
    return np.squeeze(parameter.data.cpu().numpy())

In [18]:
def graph_from_dist_tensor(dist, parameter, self_dist=True):
    if self_dist:
        assert dist.shape[0] == dist.shape[1], "Input is not pairwise dist matrix"

    # 将 NumPy 数组转换为 PyTorch 张量
    dist_tensor = dist.clone().detach()
    parameter_tensor = torch.tensor(parameter)  # 将参数也转换为 PyTorch 张量

    # 在进行比较操作时，确保使用 PyTorch 的方法
    g = (dist_tensor <= parameter_tensor).float()

    if self_dist:
        diag_idx = torch.arange(g.shape[0])
        g[diag_idx, diag_idx] = 0

    return g

In [19]:
def gen_adj_mat_tensor(data, parameter, metric="cosine"):
    assert metric == "cosine", "Only cosine distance implemented"
    dist = cosine_distance_torch(data, data)
    g = graph_from_dist_tensor(dist, parameter, self_dist=True)
    if metric == "cosine":
        adj = 1-dist
    else:
        raise NotImplementedError
    adj = adj*g 
    adj_T = adj.transpose(0,1)
    I = torch.eye(adj.shape[0])
    adj = adj + adj_T*(adj_T > adj).float() - adj*(adj_T > adj).float()
    adj = F.normalize(adj + I, p=1)
    
    return adj

In [20]:
def gen_trte_adj_mat(X_train, adj_parameter):
    adj_metric = "cosine" # cosine distance
    adj_train = []
    adj_parameter_adaptive = cal_adj_mat_parameter(adj_parameter, X_train, adj_metric)
    adj_train = gen_adj_mat_tensor(X_train, adj_parameter_adaptive, adj_metric)
    
    return adj_train

In [21]:
def one_hot_tensor(y, num_dim):
    y_onehot = torch.zeros(y.shape[0], num_dim)  # 指定设备
    y_onehot.scatter_(1, y.view(-1,1).long(), 1)
    return y_onehot


In [22]:
def cal_sample_weight(labels, num_class, use_sample_weight=True):
    labels_np = labels.numpy()  # 将 PyTorch 张量转换为 NumPy 数组
    if not use_sample_weight:
        return np.ones(len(labels_np)) / len(labels_np)
    count = np.zeros(num_class)
    for i in range(num_class):
        count[i] = np.sum(labels_np == i)
    sample_weight = np.zeros(labels_np.shape)
    for i in range(num_class):
        sample_weight[np.where(labels_np == i)[0]] = count[i] / len(labels_np)
    return torch.FloatTensor(sample_weight)  # 将 NumPy 数组转换回 PyTorch 张量

In [23]:
def process_data(X_train, y_train, X_val, y_val, X_test, y_test):
    # 转换为 Tensor
    X_train_tensor = torch.from_numpy(X_train.astype(np.float32).values)
    y_train_tensor = torch.from_numpy(y_train.values.astype(np.float32))
    X_val_tensor = torch.from_numpy(X_val.astype(np.float32).values)
    y_val_tensor = torch.from_numpy(y_val.values.astype(np.float32))
    X_test_tensor = torch.from_numpy(X_test.astype(np.float32).values)
    y_test_tensor = torch.from_numpy(y_test.values.astype(np.float32))

    return X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, X_test_tensor, y_test_tensor

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:
# 数据预处理
Methylation_train, labels_train, Methylation_val, labels_val, Methylation_test, labels_test = process_data(Methylation_train, labels_train, Methylation_val, labels_val, Methylation_test, labels_test)
sample_weight_tr = cal_sample_weight(labels_train, num_class)

labels_train = one_hot_tensor(labels_train, num_class)
labels_test = one_hot_tensor(labels_test, num_class)
labels_val = one_hot_tensor(labels_val, num_class)

graph_Methylation_train, graph_Methylation_test, graph_Methylation_val = gen_trte_adj_mat(Methylation_train, Methylation_test, Methylation_val, adj_parameter)

In [26]:
graph_Methylation_train.shape

torch.Size([483, 483])

In [27]:
SCNA_train = torch.from_numpy(SCNA_train.astype(np.float32).values)
SCNA_test = torch.from_numpy(SCNA_test.astype(np.float32).values)
SCNA_val = torch.from_numpy(SCNA_val.astype(np.float32).values)

graph_SCNA_train, graph_SCNA_test, graph_SCNA_val = gen_trte_adj_mat(SCNA_train, SCNA_test, SCNA_val, adj_parameter)

In [28]:
graph_SCNA_train.shape

torch.Size([483, 483])

In [29]:
RNAseq_train = torch.from_numpy(RNAseq_train.astype(np.float32).values)
RNAseq_test = torch.from_numpy(RNAseq_test.astype(np.float32).values)
RNAseq_val = torch.from_numpy(RNAseq_val.astype(np.float32).values)

graph_RNAseq_train, graph_RNAseq_test, graph_RNAseq_val = gen_trte_adj_mat(RNAseq_train, RNAseq_test, RNAseq_val, adj_parameter)

In [30]:
graph_RNAseq_train.shape

torch.Size([483, 483])

In [31]:
# 将三个 245x245 矩阵合并为一个 245x245x3 矩阵
grapg_train = np.stack([graph_Methylation_train, graph_SCNA_train, graph_RNAseq_train], axis=-1)

# 将三个 245x245 矩阵合并为一个 245x245x3 矩阵
grapg_test = np.stack([graph_Methylation_test, graph_SCNA_test, graph_RNAseq_test], axis=-1)

# 将三个 245x245 矩阵合并为一个 245x245x3 矩阵
grapg_val = np.stack([graph_Methylation_val, graph_SCNA_val, graph_RNAseq_val], axis=-1)

In [32]:
graph_RNAseq_test.shape

torch.Size([151, 151])

In [33]:
graph_SCNA_test.shape

torch.Size([151, 151])

In [34]:
graph_Methylation_test.shape

torch.Size([151, 151])

In [35]:
np.save('../../DATA/BRCA/graph_train.npy', grapg_train)

In [36]:
np.save('../../DATA/BRCA/graph_test.npy', grapg_test)

In [37]:
np.save('../../DATA/BRCA/graph_val.npy', grapg_val)

In [38]:
np.save('../../DATA/BRCA/labels_test.npy', labels_test)

In [39]:
np.save('../../DATA/BRCA/labels_train.npy', labels_train)

In [40]:
np.save('../../DATA/BRCA/labels_val.npy', labels_val)